## Предобработка и выделение года с помощью regexp

In [1]:
import json
import pandas as pd
import re

In [2]:
with open('../own_data/monety_parsed.json', 'r') as f:
    items = json.loads(f.read())

In [3]:
items = pd.DataFrame(items)

In [4]:
items.shape

(10002, 3)

In [5]:
items['url'].loc[0]

u'https://www.avito.ru/moskva/kollektsionirovanie/25_rubley_futbol_chm_2018_chempionat_mira_2018_emb_911584258'

In [6]:
items['full'] = items['title'] + ' ' + items['text'].apply(lambda a: ' '.join(a))

In [7]:
items['full'] = items['full'].apply(lambda s: s.lower())

In [8]:
year_regexp = ur'\b20\d\d\b|\b19\d\d\b|\b18\d\d\b|\b17\d\d\b'

In [9]:
other_regexps = [ur'\b(\d+)\s?-\s?\d+', ur'\b(\d\d)-\d\d года', ur'\b(\d\d)г']

In [10]:
def apply_regexp(s):
    matches = re.findall(year_regexp, s)
    if not matches:
        
        for pattern in other_regexps:
            matches += re.findall(pattern, s)
#         print matches
            
    if not matches:
        return 'NO_YEAR'
    return matches[0]

In [11]:
def process_year(s):
    if s == 'NO_YEAR':
        return 0
    year = int(s)
    if year < 17:
        year = year + 2000
    elif year < 100:
        year = year + 1900
    return year

In [12]:
items['year'] = items['full'].apply(apply_regexp)

In [13]:
items['year_int'] = items['year'].apply(process_year)

## Расчет необходимых параметров

Необходимо учесть, что мы получили только 10000 объявлений

Объявления без года

In [14]:
items[items['year'] == 'NO_YEAR'].shape

(1961, 6)

Расчет пропорции

In [15]:
items_with_year = items[items['year'] != 'NO_YEAR']

In [16]:
items_with_year[items_with_year['year_int'] >= 2000].shape

(3348, 6)

In [17]:
items_with_year[items_with_year['year_int'] < 2000].shape

(4693, 6)

In [18]:
float(items_with_year[items_with_year['year_int'] < 2000].shape[0]) / items_with_year[items_with_year['year_int'] >= 2000].shape[0]

1.401732377538829